In [6]:
import os,datetime
import cv2
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import random
from pathlib import Path

SEED_XML_DIR = '/datasets/scripts/generated_xml_with_noising_obj/'
SEED_IMG_DIR = '/datasets/scripts/generated_image_with_noising_obj/'
GENE_IMG_WITH_SPECIAL_OBJ = '/datasets/scripts/generated_image_with_noising_obj/'
GENE_XML_WITH_SPECIAL_OBJ = '/datasets/scripts/generated_xml_with_noising_obj/'
SPEC_OBJ_DIR = '/datasets/scripts/img_objects/'

def IOU(bbox_a, bbox_b):
    '''
    W = min(A.RT.x, B.RT.x) - max(A.LB.x, B.LB.x) 
    H = min(A.RT.y, B.RT.y) - max(A.LB.y, B.LB.y) 
    if W <= 0 or H <= 0: 
        return 0 
    SA = (A.RT.x - A.LB.x) * (A.RT.y - A.LB.y) 
    SB = (B.RT.x - B.LB.x) * (B.RT.y - B.LB.y) 
    cross = W * H return cross/(SA + SB - cross)
    '''
    W = min(bbox_a[1], bbox_b[1]) - max(bbox_a[0], bbox_b[0]) 
    H = min(bbox_a[3], bbox_b[3]) - max(bbox_a[2], bbox_b[2]) 
    if W <= 0 or H <= 0: 
        return 0
    SA = (bbox_a[1] - bbox_a[0]) * (bbox_a[3] - bbox_a[2]) 
    SB = (bbox_b[1] - bbox_b[0]) * (bbox_b[3] - bbox_b[2])  
    cross = W * H 
    return cross/(SA + SB - cross)

def generate_new_bbox(img_size, img_obj_size):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    array_y = np.arange(int(img_size[0] - img_obj_size[0]))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    new_bbox = [random_x, random_x + img_obj_size[1], random_y, random_y + img_obj_size[0]]
    #print(new_bbox)
    return new_bbox
def get_bboxes_from_etree(etree):
    root = tree.getroot()  
    objects = root.findall('object')
    bboxes = []
    for obj in objects:
        xmlbox = obj.find('bndbox')
        b = [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
        bboxes.append(b)
    return bboxes

def check_bbox(new_bbox, bboxes):
    for bbox in bboxes:
        if(IOU(new_bbox, bbox) > 0.005):
            return False
        continue
    return True

def past_and_insert(img_obj, img_array, new_position, obj_element, etree):
    new_xmlobj = generate_new_xmlobj(obj_element, new_position, img_obj.shape[:2])
    new_xml_etree = insert_to_xml(new_xmlobj, etree)
    new_pil_img = past_obj_to_background(img_obj, img_array, new_position)
    return new_pil_img, new_xml_etree

def generate_new_xmlobj(xmlobj_old, new_position, new_size):
    element_object = ET.Element('object')
    tag_name = ET.SubElement(element_object, 'name')
    tag_name.text = xmlobj_old.find('name').text

    tag_difficult = ET.SubElement(element_object, 'difficult')
    tag_difficult.text = xmlobj_old.find('difficult').text

    element_bndbox = ET.SubElement(element_object, 'bndbox')
    tag_xmin = ET.SubElement(element_bndbox, 'xmin')
    tag_ymin = ET.SubElement(element_bndbox, 'ymin')
    tag_xmax = ET.SubElement(element_bndbox, 'xmax')
    tag_ymax = ET.SubElement(element_bndbox, 'ymax')
    tag_xmin.text = str(new_position[0])
    tag_ymin.text = str(new_position[1])
    tag_xmax.text = str(new_position[0] + new_size[1])
    tag_ymax.text = str(new_position[1] + new_size[0])
    return element_object

def insert_to_xml(xml_obj, xml_etree):
    root = xml_etree.getroot()
    root.append(xml_obj)
    return xml_etree
    
def past_obj_to_background(img_obj, img_array, position = (200, 200)):
    img = Image.fromarray(img_array)
    img_obj = Image.fromarray(img_obj)
    img.paste(img_obj, position)
    return img

def inset_obj_to_an_image_and_xml(img_obj, img, obj_element, etree):
    new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    print(new_bbox)
    bboxes = get_bboxes_from_etree(etree)
    while(not check_bbox(new_bbox, bboxes)):
        print('new_bbox not suitable, retry...')
        new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
        print(new_bbox)
    new_pil_img, new_xml_etree = past_and_insert(img_obj, img, (new_bbox[0], new_bbox[2]), obj_element, etree)
    
    return new_pil_img, new_xml_etree

def generate_new_xmlobj(xmlobj_old, new_position, new_size):
    element_object = ET.Element('object')
    tag_name = ET.SubElement(element_object, 'name')
    tag_name.text = xmlobj_old.find('name').text

    tag_difficult = ET.SubElement(element_object, 'difficult')
    tag_difficult.text = xmlobj_old.find('difficult').text

    element_bndbox = ET.SubElement(element_object, 'bndbox')
    tag_xmin = ET.SubElement(element_bndbox, 'xmin')
    tag_ymin = ET.SubElement(element_bndbox, 'ymin')
    tag_xmax = ET.SubElement(element_bndbox, 'xmax')
    tag_ymax = ET.SubElement(element_bndbox, 'ymax')
    tag_xmin.text = str(new_position[0])
    tag_ymin.text = str(new_position[1])
    tag_xmax.text = str(new_position[0] + new_size[1])
    tag_ymax.text = str(new_position[1] + new_size[0])
    return element_object

def generate_xml_info_under_object(attribut_vale_dic):
    element_object = ET.Element('object')
    if(len(attribut_vale_dic) == 0):
        return False
    for key in attribut_vale_dic.keys():
        tag_name = ET.SubElement(element_object, key)
        tag_name.text = attribut_vale_dic[key]
    return element_object
#generate_xml_info_under_object({'name':'my-class','difficult':'0'})

In [7]:
seed_xml_names = os.listdir(SEED_XML_DIR)
special_obj_cls = os.listdir(SPEC_OBJ_DIR)

classes = ['person']

for cls in special_obj_cls:
        if(not cls in classes):
            continue
            
        objs = os.listdir(SPEC_OBJ_DIR + cls)
        for obj in objs:
            print(obj)
            if(not (obj.endswith('.png') or obj.endswith('.jpg'))):
                continue

            for xml_name in seed_xml_names:
                img_name = SEED_IMG_DIR + xml_name[:-3] + 'jpg'
                xml_name = SEED_XML_DIR + xml_name
                print(img_name)
                print(xml_name)
                tree=ET.parse(xml_name)
                img = cv2.imread(img_name)
                img_obj = cv2.imread(SPEC_OBJ_DIR + cls + '/' + obj)
                if((img.shape[0] < img_obj.shape[0]) or (img.shape[1] < img_obj.shape[1]) or 
                   ((img.shape[0]*img.shape[1])/(img_obj.shape[0]*img_obj.shape[1]) < 4)):
                    continue
                dic_ = {'name':cls, 'difficult':'0'}
                obj_element = generate_xml_info_under_object(dic_)
                new_pil_img, new_xml_etree = inset_obj_to_an_image_and_xml(img_obj, img, obj_element, tree)
                print('insert success')
                img = np.array(new_pil_img)

                ## save xml and img
                print('save image and xml')
                time_mark = datetime.datetime.now()
                time_str = time_mark.strftime("%Y%m%d%H%M%S_")
                new_pil_img = Image.fromarray(img[:,:,(2, 1, 0)])
                new_pil_img.save(GENE_IMG_WITH_SPECIAL_OBJ + img_name.split('/')[-1])
                new_xml_etree.write(GENE_XML_WITH_SPECIAL_OBJ + xml_name.split('/')[-1])
                #break

00000303093_2.jpg
/datasets/scripts/generated_image_with_noising_obj/20181219173506608100noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/20181219173506608100noising.xml
[152, 180, 431, 512]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/2018121917350769590320181219173507063972noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/2018121917350769590320181219173507063972noising.xml
[901, 929, 67, 148]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/2018121917350769590320181219173507179831noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/2018121917350769590320181219173507179831noising.xml
[209, 237, 70, 151]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/2018121917350769590320181219173503809027noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/2018121917350769590320181219173503809027noising.xml
[676, 704, 398, 479]
new_bbox not suitable,

/datasets/scripts/generated_image_with_noising_obj/20181219173504754835noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/20181219173504754835noising.xml
[283, 311, 53, 134]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/201812191735076959032018121917350769590320181219173503974885noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/201812191735076959032018121917350769590320181219173503974885noising.xml
[833, 861, 35, 116]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/20181219173502419248noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/20181219173502419248noising.xml
[303, 331, 264, 345]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/2018121917350769590320181219173503533941noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/2018121917350769590320181219173503533941noising.xml
[387, 415, 122, 203]
new_bbox not suitable, retry...
[819, 84

insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/2018121917350769590320181219173504754835noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/2018121917350769590320181219173504754835noising.xml
[375, 403, 26, 107]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/201812191735076959032018121917350769590320181219173507465179noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/201812191735076959032018121917350769590320181219173507465179noising.xml
[286, 314, 425, 506]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/20181219173505046979noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/20181219173505046979noising.xml
[496, 524, 116, 197]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/2018121917350769590320181219173504359756noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/20181219173507695903201812191735043597

insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/20181219173502693613noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/20181219173502693613noising.xml
[14, 42, 532, 613]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/20181219173507179831noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/20181219173507179831noising.xml
[554, 582, 503, 584]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/2018121917350769590320181219173503864095noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/2018121917350769590320181219173503864095noising.xml
[75, 103, 81, 162]
new_bbox not suitable, retry...
[35, 63, 53, 134]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/20181219173504698432noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/20181219173504698432noising.xml
[622, 650, 250, 331]
new_bbox not suitable, retry...
[495, 

/datasets/scripts/generated_image_with_noising_obj/20181219173506089659noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/20181219173506089659noising.xml
[824, 852, 401, 482]
new_bbox not suitable, retry...
[941, 969, 89, 170]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/20181219173506892023noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/20181219173506892023noising.xml
[91, 119, 224, 305]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/201812191735076959032018121917350769590320181219173504083590noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/201812191735076959032018121917350769590320181219173504083590noising.xml
[292, 320, 279, 360]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/201812191735076959032018121917350769590320181219173505593896noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/2018121917350769590320181219173507695903

...

save image and xml
/datasets/scripts/generated_image_with_noising_obj/2018121917350769590320181219173506033939noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/2018121917350769590320181219173506033939noising.xml
[565, 598, 492, 623]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/201812191735076959032018121917350769590320181219173506777668noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/201812191735076959032018121917350769590320181219173506777668noising.xml
[426, 459, 248, 379]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/201812191735076959032018121917350769590320181219173503588143noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/201812191735076959032018121917350769590320181219173503588143noising.xml
[611, 644, 479, 610]
new_bbox not suitable, retry...
[977, 1010, 289, 420]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/2018121917350769590

[444, 477, 144, 275]
new_bbox not suitable, retry...
[869, 902, 287, 418]
new_bbox not suitable, retry...
[986, 1019, 432, 563]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/20181219173507234563noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/20181219173507234563noising.xml
[18, 51, 263, 394]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/201812191735076959032018121917350769590320181219173507639719noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/201812191735076959032018121917350769590320181219173507639719noising.xml
[212, 245, 416, 547]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/20181219173502473942noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/20181219173502473942noising.xml
[912, 945, 40, 171]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/20181219173507695903201812191735076959032018121917350

/datasets/scripts/generated_image_with_noising_obj/20181219173502966251noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/20181219173502966251noising.xml
[493, 526, 210, 341]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/201812191735076959032018121917350769590320181219173503864095noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/201812191735076959032018121917350769590320181219173503864095noising.xml
[521, 554, 371, 502]
new_bbox not suitable, retry...
[944, 977, 243, 374]
new_bbox not suitable, retry...
[959, 992, 187, 318]
new_bbox not suitable, retry...
[82, 115, 479, 610]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/201812191735076959032018121917350769590320181219173507695903noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/201812191735076959032018121917350769590320181219173507695903noising.xml
[124, 157, 225, 356]
insert success
save image and xml
/datasets/scripts/generat

/datasets/scripts/generated_image_with_noising_obj/2018121917350769590320181219173504931347noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/2018121917350769590320181219173504931347noising.xml
[660, 693, 131, 262]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/201812191735076959032018121917350769590320181219173503140590noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/201812191735076959032018121917350769590320181219173503140590noising.xml
[476, 509, 281, 412]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/2018121917350769590320181219173506951175noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/2018121917350769590320181219173506951175noising.xml
[707, 740, 50, 181]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/20181219173505649678noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/20181219173505649678noising.xml
[314, 347, 58, 189]


/datasets/scripts/generated_image_with_noising_obj/2018121917350769590320181219173506089659noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/2018121917350769590320181219173506089659noising.xml
[361, 394, 26, 157]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/20181219173507289591noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/20181219173507289591noising.xml
[470, 503, 362, 493]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/20181219173504083590noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/20181219173504083590noising.xml
[672, 705, 115, 246]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/2018121917350769590320181219173505267203noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/2018121917350769590320181219173505267203noising.xml
[666, 699, 376, 507]
insert success
save image and xml
/datasets/scripts/generated_image_with_noisin